Этот проект был сделан на коллабе, поэтому тут в течение первого экран код загрузки файликов, который запускать НЕ НАДО. Плюс я не стал класть данные в реп (бо они стандартны), но в тегах находится та информация, которую надо загрузить.

In [0]:
from google.colab import files
uploaded = files.upload()
#meta.json

Saving meta.json to meta.json


In [0]:
from google.colab import files
uploaded = files.upload()
#model.hdf5

Saving model.hdf5 to model.hdf5


In [0]:
from google.colab import files
uploaded = files.upload()
#options.json

Saving options.json to options.json


In [0]:
from google.colab import files
uploaded = files.upload()
#vocab.txt.gz

Saving vocab.txt.gz to vocab.txt.gz


In [0]:
from google.colab import files
uploaded = files.upload()
#elmo_helpers.py

Saving elmo_helpers.py to elmo_helpers.py


In [0]:
from google.colab import files
uploaded = files.upload()
#bilm.zip

Saving bilm.zip to bilm.zip


In [0]:
import zipfile
with zipfile.ZipFile('bilm.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [0]:
from google.colab import files
uploaded = files.upload()
#quora-question-pairs-russian.zip

Saving quora-question-pairs-russian.zip to quora-question-pairs-russian.zip


In [0]:
with zipfile.ZipFile('quora-question-pairs-russian.zip', 'r') as zip_ref:
    zip_ref.extractall()

**Начало программы**

In [85]:
%load_ext autoreload

import time
import numpy as np
import tensorflow as tf
from elmo_helpers import tokenize, get_elmo_vectors, load_elmo_embeddings

tf.reset_default_graph()
elmo_path = '.'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
#!pip install pymorphy2

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 7.1MB 10.1MB/s 


In [0]:
import csv
import pymorphy2
import string
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial

morph = pymorphy2.MorphAnalyzer()

In [0]:
def get_texts():
    corpus = []
    inquiery = []
    scores = []
    normal = []

    with open('quora_question_pairs_rus.csv', encoding = 'utf-8') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            normal.append(row[2])
            text = preproc(row[2])
            corpus.append(text)
            inquiery.append(row[1])
            #lenghths.append(len(text.split(' ')))
            scores.append(row[3])
            if len(scores) == 20000:
                #return [corpus, inquiery, lenghths, scores, normal]
                return [corpus, inquiery, scores, normal]

def preproc(data):                      # препроцессинг (убираем знаки препинания и числа, приводим всё к начальной форме)
            data = data.split()        
            text = ''                   # числа есть только в тех информации, в самом скрипте все числа пишутся буквами
            for word in data:
                word = word.strip('[!,.?"]')
                p = morph.parse(word.strip())[0]
                p = p.normal_form
                if p != '-':
                    if re.search(r'\d', p) == None:
                        text  = text + ' ' + p
            return text[1:]    

In [0]:
data = get_texts()

In [0]:
corpus_texts, inquiery_texts, scores, normal_texts =  data[0][1:], data[1][1:], data[2][1:], data[3][1:]
del data

In [90]:
print(corpus_texts[1])
print(inquiery_texts[1])
print(scores[1])

как повысить скорость интернет путём взлом через dns
как я могу увеличить скорость моего интернет-соединения, используя vpn
0


In [91]:
batcher, sentence_character_ids, elmo_sentence_input = load_elmo_embeddings(elmo_path)

In [0]:
vecs = []

Тут в глубинной реализации где-то затерялся принт - я не стал его убирать

In [0]:
for i in range(2000):
  raw_sentences = [corpus_texts[i]]

  sentences = [tokenize(s) for s in raw_sentences]
  
  with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    elmo_vectors = get_elmo_vectors(
        sess, sentences, batcher, sentence_character_ids, elmo_sentence_input)
    
    for vect in elmo_vectors:
        res = np.zeros(len(vect[0]))
        for j in vect:
          res += j
          res = res/len(vect)
          vecs.append(res)
        

Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
S

In [97]:
vecs

[array([-0.93083562, -0.17554013,  0.37956826, ..., -0.19400053,
        -0.00214151,  0.18739642]),
 array([-0.74837598,  0.02034614,  0.02130339, ..., -0.55606254,
         0.74399674,  0.46763022]),
 array([-0.03070219,  0.08528612,  0.22438321, ..., -0.29780649,
        -0.81957913, -0.32135442]),
 array([ 0.68198647, -0.36929756,  0.11897979, ..., -0.29824377,
        -0.55085511, -0.38918823]),
 array([ 0.39191033, -0.02581573, -0.12583895, ..., -0.43406862,
         0.17119677, -0.39344559]),
 array([ 2.67914763e-01, -8.62421848e-02,  2.19218427e-01, ...,
        -5.13998563e-01,  2.31383126e-01,  2.66631011e-04]),
 array([ 0.48260039, -0.28767835,  0.63970414, ..., -0.91401798,
         0.25399959,  0.14920396]),
 array([ 0.07920985,  0.03247223,  0.37868453, ...,  0.25485267,
        -0.24169361, -0.25554459]),
 array([ 0.29482185, -0.28212571,  0.09198341, ...,  0.13257032,
         0.1612001 ,  0.02601013]),
 array([ 0.58924763, -0.11066775,  0.15910328, ..., -0.17696937,
  

In [0]:
def inquiery_vector(inq ):
    inqiery = preproc(inq) 
    raw_sentences = [inq]
    sentences = [tokenize(s) for s in raw_sentences]
    with tf.Session() as sess:
    
      sess.run(tf.global_variables_initializer())
    
      elmo_vectors = get_elmo_vectors(
        sess, sentences, batcher, sentence_character_ids, elmo_sentence_input)
    
    
      for vect in elmo_vectors:
        res = np.zeros(len(vect[0]))
        for j in vect:
          res += j
          res = res/len(vect)
          
    return res

In [0]:
vecz = np.nan_to_num(vecs)

In [0]:
def search_elmo(inq):
    
    inq_vec = inquiery_vector(inq).reshape(-1,1)
    results = {}
    for index, text in enumerate(vecz):
        a = text.reshape(-1,1)
        res = 1 - spatial.distance.cosine(a, inq_vec )
        results[index] = res
    r = sorted(results, key=results.get, reverse=True)[:5]
    #r = sorted(results, key=results.get, reverse=True)
    for i in r:
        print(normal_texts[i])
    return 

In [101]:
search_elmo('введи слово')

Sentences in this batch: 1


IndexError: ignored

В общем моё elmo демонстрирует самый убогий  результат из всего, что я пробовал